<a href="https://colab.research.google.com/github/yellowlarva-sketch/aml-fraud-detection/blob/main/01_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IBM Transactions for Anti Money Laundering

We aim to develop a predictive model that identifies potential money laundering cases by analyzing patterns in historical account transaction data.

Import Data from Kaggle Project
https://www.kaggle.com/datasets/ealtman2019/ibm-transactions-for-anti-money-laundering-aml/code

In [1]:
# Connect drive to save data
from google.colab import drive
drive.mount('/content/drive')

# Set directory
main_dir = '/content/drive/My Drive/Colab Notebooks/'
input_dir = main_dir + '/00_inputs/'
script_dir = main_dir + '/01_scripts/'
output_dir = main_dir + '/02_outputs/'
non_predictive_cols = ['target', 'date']

# Import libraries
!pip install scorecardpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from scorecardpy import woebin_plot, woebin
import warnings
warnings.filterwarnings("ignore")
import os
os.chdir(script_dir)

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scorecardpy: filename=scorecardpy-0.1.9.7-py3-none-any.whl size=60629 sha256=8a18cb0a970a35b26dc6d9cf40bf31422065c4c36436aede55fed588243bc648
  Stored in directory: /root/.cache/pip/wheels/68/88/65/58d6aa058698c47d7a24352952ea07a1409cc2eff3a8087c0c
Successfully built scorecardpy


In [2]:
# Import data
df = pd.read_csv(input_dir + '/HI-Small_Trans.csv')
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


In [3]:
# Set column name
id_col = 'Account'
target = 'Is Laundering'

print("Number of transactions:", len(df), df.shape)
print("Number of unique account ID:", df[id_col].nunique())
print("Number of counterparty account ID:", df['Account.1'].nunique())
print("Date range:", min(df['Timestamp']), max(df['Timestamp']))

Number of transactions: 5078345 (5078345, 11)
Number of unique account ID: 496995
Number of counterparty account ID: 420636
Date range: 2022/09/01 00:00 2022/09/18 16:18


In [4]:
# Daily volume --> Out of Time sample start from 2022/09/08
df['Timestamp'].str[:10].value_counts(normalize=True).sort_index()

,proportion
Timestamp,
2022/09/01,0.219544
2022/09/02,0.148562
2022/09/03,0.040837
2022/09/04,0.040846
2022/09/05,0.095041
2022/09/06,0.094930
2022/09/07,0.095061
2022/09/08,0.095065
2022/09/09,0.128874


In [5]:
pd.concat([df[target].value_counts(),
           df[target].value_counts(normalize=True)]
          , axis=1)

,count,proportion
Is Laundering,,
0,5073168,0.998981
1,5177,0.001019


In [6]:
df['Payment Format'].value_counts()

,count
Payment Format,
Cheque,1864331
Credit Card,1323324
ACH,600797
Cash,490891
Reinvestment,481056
Wire,171855
Bitcoin,146091


In [7]:
df['Payment Currency'].value_counts()

,count
Payment Currency,
US Dollar,1895172
Euro,1168297
Swiss Franc,234860
Yuan,213752
Shekel,192184
Rupee,190202
UK Pound,180738
Yen,155209
Ruble,155178


In [8]:
df['Receiving Currency'].value_counts()

,count
Receiving Currency,
US Dollar,1879341
Euro,1172017
Swiss Franc,237884
Yuan,206551
Shekel,194988
Rupee,192065
UK Pound,181255
Ruble,157361
Yen,156319


In [9]:
# Transaction level calculation
df_1 = pd.get_dummies(df, columns=['Payment Format','Payment Currency','Receiving Currency'])
df_1['self_transfer'] = df_1['Account'] == df_1['Account.1']
df_1['biz_dt'] = df_1['Timestamp'].str[:10]

In [10]:
# Summarize predict level based on Account and Transaction Date
df_modeling = pd.concat([
    df_1.groupby(['biz_dt','Account'])[[i for i in df_1.columns if i not in ['biz_dt','Timestamp','From Bank','Account','To Bank','Account.1']]].sum(),
    df_1.groupby(['biz_dt','Account'])['Account.1'].nunique()
], axis=1)
df_modeling.reset_index(inplace=True)
df_modeling.head()

,biz_dt,Account,Amount Received,Amount Paid,Is Laundering,Payment Format_ACH,Payment Format_Bitcoin,Payment Format_Cash,Payment Format_Cheque,Payment Format_Credit Card,...,Receiving Currency_Rupee,Receiving Currency_Saudi Riyal,Receiving Currency_Shekel,Receiving Currency_Swiss Franc,Receiving Currency_UK Pound,Receiving Currency_US Dollar,Receiving Currency_Yen,Receiving Currency_Yuan,self_transfer,Account.1
0,2022/09/01,100428660,1.348251e+10,1.348251e+10,37,0,0,4852,10999,6950,...,0,0,0,0,0,22801,0,0,0,10912
1,2022/09/01,1004286A8,8.463515e+09,8.463515e+09,23,0,0,2845,6737,4257,...,0,0,0,0,0,0,0,0,0,6701
2,2022/09/01,1004286F0,4.014691e+09,4.014691e+09,9,0,0,543,1223,740,...,0,0,0,0,0,0,0,2506,0,1223
3,2022/09/01,100428738,9.251903e+10,9.251903e+10,8,0,0,401,871,559,...,0,0,0,0,0,0,1831,0,0,867
4,2022/09/01,100428780,1.302256e+11,1.302256e+11,3,0,0,497,1097,702,...,2296,0,0,0,0,0,0,0,0,1099


In [11]:
df_modeling['Is Laundering'] = np.where(df_modeling['Is Laundering'] >= 1, 1, 0)
df_modeling['Is Laundering'].value_counts()

,count
Is Laundering,
0,1546751
1,3965


In [12]:
# Split Train and Test for model development sample
cutoff_date = '2022/09/08'
oot = df_modeling[df_modeling['biz_dt'] >= cutoff_date].set_index(['biz_dt','Account'])[[i for i in df_modeling.columns if i not in ['biz_dt','Account']]]

RANDOM_SEED = 2025
train,test = train_test_split(df_modeling[df_modeling['biz_dt'] < cutoff_date].set_index(['biz_dt','Account'])[[i for i in df_modeling.columns if i not in ['biz_dt','Account']]], test_size=0.2, random_state=RANDOM_SEED)
train,valid = train_test_split(train, test_size=0.2, random_state=RANDOM_SEED)

In [13]:
print('Train:', train.shape)
print('Test:', test.shape)
print('Valid:', valid.shape)
print('OOT:', oot.shape)

Train: (734352, 42)
Test: (229486, 42)
Valid: (183588, 42)
OOT: (403290, 42)


In [14]:
train['X_fold'] = 'train'
test['X_fold'] = 'test'
valid['X_fold'] = 'valid'
oot['X_fold'] = 'outoftime'

mds = pd.concat([train, test, valid, oot])
mds[target].value_counts()

,count
Is Laundering,
0,1546751
1,3965


In [15]:
mds.groupby('X_fold')[target].value_counts().unstack()

Is Laundering,0,1
X_fold,,
outoftime,401639,1651
test,229055,431
train,732837,1515
valid,183220,368


In [16]:
features = [i for i in mds.columns if i not in ['biz_dt','Account','X_fold',target]]

pd.Series(features).to_csv(output_dir + 'predictive_columns.csv', index=False)

In [17]:
mds = mds.reset_index()
mds.to_csv(output_dir + 'modeling_data.csv', index=False)

END